# Imports


In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import pandas as pd
import matplotlib.pyplot as plt



/Users/nicolehao/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Loading

In [10]:
import time
from sklearn.model_selection import train_test_split
# from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

# Loading

In [11]:
# Load the second uploaded notebook

notebook_path_2 = "Parallel CNN-Based Transient Stability Analysis.ipynb"
with open(notebook_path_2, 'r', encoding='utf-8') as f:
    notebook_2 = nbformat.read(f, as_version=4)

# Extract code cells from the second notebook
code_cells_2 = [cell['source'] for cell in notebook_2.cells if cell.cell_type == 'code']
code_cells_2[:3]  # Show first few code cells to understand the structure


FileNotFoundError: [Errno 2] No such file or directory: 'Parallel CNN-Based Transient Stability Analysis.ipynb'


# Scaling Test

In [3]:
# Scaling Test (Data Size Only)

def build_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Split data once to reuse same test set
# Split data
X_train_full, X_test, y_train_full, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

fractions = [0.1, 0.25, 0.5, 0.75, 1.0]
results = []

for frac in fractions:
    size = int(len(X_train_full) * frac)
    X_train = X_train_full[:size]
    y_train = y_train_full[:size]

    clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=42)

    start_time = time.time()
    clf.fit(X_train, y_train)
    duration = time.time() - start_time

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    results.append((frac, size, duration, acc))
    print(f"Data Fraction: {frac:.2f}, Train Size: {size}, Time: {duration:.2f}s, Accuracy: {acc:.4f}")

# Results
df_results = pd.DataFrame(results, columns=["Fraction", "Train Size", "Time (s)", "Accuracy"])

# Plot
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Training Size')
ax1.set_ylabel('Training Time (s)', color=color)
ax1.plot(df_results["Train Size"], df_results["Time (s)"], marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Accuracy', color=color)
ax2.plot(df_results["Train Size"], df_results["Accuracy"], marker='o', color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title("Scaling Test: MLP Training Time & Accuracy")
plt.show()

df_results


NameError: name 'data' is not defined

In [ ]:
# Display results as DataFrame
df_results = pd.DataFrame(results, columns=["Fraction", "Train Size", "Time (s)", "Accuracy"])
print(df_results)


In [ ]:
# Plot
df_results.plot(x="Train Size", y=["Time (s)", "Accuracy"], marker='o', title="Scaling Test Results")
plt.grid(True)
plt.show()
